# Epistasis 

## Aim

This notebook uses plink2 option to evaluate epistasis (gene-gene interaction)

In [ ]:
[global]
# the output directory for generated files
parameter: cwd = path
# Genotype files in plink binary file
parameter: bfile = path
# Specific phenotype file for analysis FID, IID and affection status
parameter: phenoFile =path
# For cluster jobs, number commands to run per job
parameter: job_size = 1
# Specific number of threads to use
parameter: numThreads = 10
#Load the plink module in the cluster
parameter: plink_module = '''
        module load PLINK/1.90-beta5.3 
        echo "Module plink loaded"
        {cmd}'''

In [ ]:
[maf]
input: bfile
output: f'{cwd}/{phenoFile:bn}.maf.frq.cc'
task: trunk_workers = 1, walltime = '10h', mem = '20G', cores = numThreads, tags = f'{step_name}_{_output[0]:bn}'
bash: expand= "${ }", stderr = f'{_output[0]:n}.stderr', stdout = f'{_output[0]:n}.stdout', template = '{cmd}' if executable('plink').target_exists() else plink_module
    plink \
      --bfile ${bfile:n} \
      --pheno ${phenoFile} --1\
      --freq case-control \
      --threads ${numThreads} \
      --out ${_output[0]:nn} 

In [ ]:
[epistasis]
# The file with the interactions to be analyzed
parameter: setFile = path
input: bfile, setFile
output: f'{cwd}/{phenoFile:bn}.epi.cc'
task: trunk_workers = 1, walltime = '15h', mem = '40G', cores = numThreads, tags = f'{step_name}_{_output[0]:bn}'
bash: expand= "${ }", stderr = f'{_output[0]:n}.stderr', stdout = f'{_output[0]:n}.stdout', template = '{cmd}' if executable('plink').target_exists() else plink_module
    plink \
      --bfile ${bfile:n} \
      --pheno ${phenoFile} --1\
      --epistasis set-by-set \
      --epi1 0.05 \
      --set-test\
      --set ${setFile} \
      --threads ${numThreads} \
      --out ${_output[0]:nn} 

In [ ]:
[fast_epistasis]
input: bfile
output: f'{cwd}/{phenoFile:bn}.fast.epi.cc'
task: trunk_workers = 1, walltime = '15h', mem = '80G', cores = numThreads, tags = f'{step_name}_{_output[0]:bn}'
bash: expand= "${ }", stderr = f'{_output[0]:n}.stderr', stdout = f'{_output[0]:n}.stdout', template = '{cmd}' if executable('plink').target_exists() else plink_module
    plink \
      --bfile ${bfile:n} \
      --pheno ${phenoFile} --1\
      --fast-epistasis boost \
      --epi1 5e-06 \
      --epi2 5e-06 \
      --threads ${numThreads} \
      --out ${_output[0]:nn} 